<a href="https://colab.research.google.com/github/alfitranurr/DATA-MINING/blob/main/Tugas_1_(Data_Teks)_Modul_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.2 MB/s eta 0:00:00


In [5]:
pip install nltk

# 1. Persiapan Data

In [8]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import emoji
import warnings
warnings.filterwarnings('ignore')

# Download resource NLTK
nltk.download('punkt_tab')
nltk.download('stopwords')

# Membaca dataset
data = pd.read_csv('Tugas 1_NIM Genap_Modul 4.csv')

# Melihat sekilas dataset
print(data.head())
print(data.info())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  label                                               text
0  REAL  Payal has accused filmmaker Anurag Kashyap of ...
1  FAKE  A four-minute-long video of a woman criticisin...
2  FAKE  Republic Poll, a fake Twitter account imitatin...
3  REAL  Delhi teen finds place on UN green list, turns...
4  REAL  Delhi: A high-level meeting underway at reside...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3729 entries, 0 to 3728
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   3729 non-null   object
 1   text    3721 non-null   object
dtypes: object(2)
memory usage: 58.4+ KB
None


# 2. Data Cleaning

In [9]:
# Fungsi untuk membersihkan teks
def clean_text(text):
    # Konversi ke huruf kecil
    text = str(text).lower()

    # Menghapus emoticon
    text = emoji.replace_emoji(text, replace='')

    # Mengganti tanda baca seperti - dengan spasi
    text = text.replace('-', ' ')

    # Menghapus karakter non-alfanumerik kecuali spasi
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Menghapus URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Menghapus angka
    text = re.sub(r'\d+', '', text)

    # Tokenisasi
    tokens = word_tokenize(text)

    # Menghapus stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Menggabungkan kembali token
    text = ' '.join(tokens)

    # Menghapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Menerapkan pembersihan pada kolom teks
data['cleaned_text'] = data['text'].apply(clean_text)

# Memeriksa hasil pembersihan
print(data[['text', 'cleaned_text']].head())

                                                text  \
0  Payal has accused filmmaker Anurag Kashyap of ...   
1  A four-minute-long video of a woman criticisin...   
2  Republic Poll, a fake Twitter account imitatin...   
3  Delhi teen finds place on UN green list, turns...   
4  Delhi: A high-level meeting underway at reside...   

                                        cleaned_text  
0  payal accused filmmaker anurag kashyap behavin...  
1  four minute long video woman criticising gover...  
2  republic poll fake twitter account imitating a...  
3  delhi teen finds place un green list turns gla...  
4  delhi high level meeting underway residence ra...  


# 3. Train Test Split

In [10]:
# Mengencode label (REAL=1, FAKE=0)
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label'])

# Memisahkan fitur dan label
X = data['cleaned_text']
y = data['label_encoded']

# Membagi data menjadi train dan test (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Memeriksa distribusi data
print(f"Ukuran X_train: {X_train.shape}")
print(f"Ukuran X_test: {X_test.shape}")
print(f"Distribusi label di y_train: {np.bincount(y_train)}")
print(f"Distribusi label di y_test: {np.bincount(y_test)}")

Ukuran X_train: (2983,)
Ukuran X_test: (746,)
Distribusi label di y_train: [1501 1482]
Distribusi label di y_test: [376 370]


# 4. Vektorisasi Teks

In [11]:
# Inisialisasi TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

# Fit dan transform pada data pelatihan
X_train_tfidf = tfidf.fit_transform(X_train)

# Transform pada data pengujian
X_test_tfidf = tfidf.transform(X_test)

# Memeriksa bentuk matriks
print(f"Bentuk X_train_tfidf: {X_train_tfidf.shape}")
print(f"Bentuk X_test_tfidf: {X_test_tfidf.shape}")

Bentuk X_train_tfidf: (2983, 5000)
Bentuk X_test_tfidf: (746, 5000)


# 5. Model Naive Bayes

In [12]:
# Inisialisasi model Naive Bayes
nb_model = MultinomialNB()

# Melatih model
nb_model.fit(X_train_tfidf, y_train)

# Prediksi pada data pengujian
y_pred_nb = nb_model.predict(X_test_tfidf)

# Evaluasi model
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Akurasi Naive Bayes: {accuracy_nb:.4f}")
print("\nClassification Report Naive Bayes:")
print(classification_report(y_test, y_pred_nb, target_names=label_encoder.classes_))
print("\nConfusion Matrix Naive Bayes:")
print(confusion_matrix(y_test, y_pred_nb))

Akurasi Naive Bayes: 0.9504

Classification Report Naive Bayes:
              precision    recall  f1-score   support

        FAKE       0.91      1.00      0.95       376
        REAL       1.00      0.90      0.95       370

    accuracy                           0.95       746
   macro avg       0.96      0.95      0.95       746
weighted avg       0.95      0.95      0.95       746


Confusion Matrix Naive Bayes:
[[376   0]
 [ 37 333]]


# 6. Model ANN

In [13]:
# Mengubah data ke format dense untuk ANN
X_train_dense = X_train_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()

# Membangun model ANN
ann_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Kompilasi model
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Melatih model
history = ann_model.fit(X_train_dense, y_train, epochs=10, batch_size=32,
                       validation_data=(X_test_dense, y_test), verbose=1)

# Evaluasi model
loss, accuracy_ann = ann_model.evaluate(X_test_dense, y_test)
print(f"Akurasi ANN: {accuracy_ann:.4f}")

# Prediksi untuk classification report
y_pred_ann = (ann_model.predict(X_test_dense) > 0.5).astype(int)
print("\nClassification Report ANN:")
print(classification_report(y_test, y_pred_ann, target_names=label_encoder.classes_))
print("\nConfusion Matrix ANN:")
print(confusion_matrix(y_test, y_pred_ann))

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.8995 - loss: 0.4600 - val_accuracy: 0.9946 - val_loss: 0.0170
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.9975 - loss: 0.0130 - val_accuracy: 0.9960 - val_loss: 0.0102
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.9984 - loss: 0.0059 - val_accuracy: 0.9960 - val_loss: 0.0080
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.9987 - loss: 0.0028 - val_accuracy: 0.9960 - val_loss: 0.0069
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9980 - loss: 0.0036 - val_accuracy: 0.9946 - val_loss: 0.0063
Epoch 6/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9981 - loss: 0.0030 - val_accuracy: 0.9973 - val_loss: 0.0060
Epoch 7/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9987 - loss: 0.0018 - val_accuracy: 0.9973 - val_loss: 0.0059
Epoch 8/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9993 - loss: 0.0019 - val_accuracy: 0.9973 - v

# 7. Perbandingan Akurasi

In [15]:
from tabulate import tabulate

# Perbandingan Akurasi
print("\nPerbandingan Akurasi:")
print(f"Akurasi Naive Bayes: {accuracy_nb:.4f}")
print(f"Akurasi ANN: {accuracy_ann:.4f}")
print(f"Perbedaan Akurasi (ANN - Naive Bayes): {(accuracy_ann - accuracy_nb):.4f}")

# Menentukan model terbaik
best_model = "Naive Bayes" if accuracy_nb > accuracy_ann else "ANN"
print(f"Model terbaik berdasarkan akurasi: {best_model}")

# Membuat tabel
table_data = [
    ["Naive Bayes", f"{accuracy_nb:.4f}", f"{(accuracy_ann - accuracy_nb):.4f}", best_model if accuracy_nb > accuracy_ann else ""],
    ["ANN", f"{accuracy_ann:.4f}", "", best_model if accuracy_ann >= accuracy_nb else ""]
]

headers = ["Model", "Akurasi", "Perbedaan Akurasi (ANN - Naive Bayes)", "Model Terbaik"]

# Menampilkan tabel
print("\nTabel Perbandingan Akurasi:")
print(tabulate(table_data, headers=headers, tablefmt="grid"))


Perbandingan Akurasi:
Akurasi Naive Bayes: 0.9504
Akurasi ANN: 0.9946
Perbedaan Akurasi (ANN - Naive Bayes): 0.0442
Model terbaik berdasarkan akurasi: ANN

Tabel Perbandingan Akurasi:
+-------------+-----------+-----------------------------------------+-----------------+
| Model       |   Akurasi | Perbedaan Akurasi (ANN - Naive Bayes)   | Model Terbaik   |
+=============+===========+=========================================+=================+
| Naive Bayes |    0.9504 | 0.0442                                  |                 |
+-------------+-----------+-----------------------------------------+-----------------+
| ANN         |    0.9946 |                                         | ANN             |
+-------------+-----------+-----------------------------------------+-----------------+


# 8. Contoh Prediksi

In [17]:
# Teks bebas untuk prediksi
sample_text = "A viral video claims that the government is hiding the truth about a new disease outbreak in the capital."

# Membersihkan teks
cleaned_sample = clean_text(sample_text)

# Vektorisasi teks
sample_tfidf = tfidf.transform([cleaned_sample])
sample_tfidf_dense = sample_tfidf.toarray()

# Prediksi dengan Naive Bayes
prediction_nb = nb_model.predict(sample_tfidf)
predicted_label_nb = label_encoder.inverse_transform(prediction_nb)[0]

# Prediksi dengan ANN
prediction_ann = (ann_model.predict(sample_tfidf_dense) > 0.5).astype(int)
predicted_label_ann = label_encoder.inverse_transform(prediction_ann.flatten())[0]

# Menampilkan hasil prediksi
print("\nContoh Prediksi:")
print(f"Teks: {sample_text}")
print(f"Prediksi Naive Bayes: {predicted_label_nb}")
print(f"Prediksi ANN: {predicted_label_ann}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step

Contoh Prediksi:
Teks: A viral video claims that the government is hiding the truth about a new disease outbreak in the capital.
Prediksi Naive Bayes: FAKE
Prediksi ANN: FAKE
